## Removal of pig and human sequences:
Pig and human metagenomes contain host contamination (most likely). 
We need to at least make a not of what % of each dataset is contamination, and ideally remove contamination from each before using it for other downstream analysis. 

Have tried sig subtract , but in order to use that I needed to flatten the signatures, which I think messed with downstream analysis.

I would like to write a conscise snakefile that for each signature removes these kmers and outputs a gather csv with f_orig_query (perc of metaG that matches contaminant genome). 

Snakefile needs to be able to take in lists of SRA numbers, and based off that put output in the correct folder (pig or human, maybe also distinguish between yes/no mags built). And for input files there are different paths as well. 

Steps:
1. fastmultigather metaG x  pig genome
2. fastmultigather metaG x  human genome
3. concat these output files
4. gather x metagenome with output from 3 as a picklist
5. use --output-unassigned and save as zip. 
6. Save a csv output as well of gather matches. 

### Fastmultigather x genomes
- Looks like i did this for all processed pig metaGs (still better than nothing)

Need to do this for pig metaGs without MAGs built. and for the human metaGs. 



In [ ]:
# flattening and then subtracting doesn't seem to work:
# Maybe do fmg, then use the result as a picklist for gather, and remove everything host/pig related
# treshold it?? (default is 50kb) probs keep t at 0
sourmash scripts fastmultigather \
../siglist.txt /group/ctbrowngrp2/scratch/annie/2024-pigparadigm/resources/sus_scrofa/susScr11.sig.gz \
-k 21 -s 1000 -c 40 -t 0

# same for human genome (but do in a diff folder)
mamba activate branchwater
sourmash scripts fastmultigather \
../siglist.txt /group/ctbrowngrp/sourmash-db/hg38/hg38-entire.sig.zip \
-k 21 -s 1000 -c 50 -t 0

In [ ]:
# Run snakefile that concatenates fmg output, then uses that as a picklist
# srun snakemake
srun --account=ctbrowngrp -p med2 -J fmg -t 5:00:00 -c 50 --mem=50gb --pty bash
mamba activate branchwater

snakemake -s Snakefile_pig --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 40 --rerun-incomplete -k -n

In [ ]:
# human db =  /group/ctbrowngrp/sourmash-db/hg38/hg38-entire.sig.zip
# pigdb = '/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/resources/sus_scrofa/susScr11.sig.gz'


## Host removal other signatures
- 100 pig metaGs that are not used for MAGs (read_pig_nomag)
- 100 human metaGs. (read_human_hostrm)

Do fmg in the same folder


In [ ]:
# removed human and pig stuff
